In [ ]:
# @title Import packages
import numpy as np
import matplotlib.pyplot as plt

- Basic setup to simulate trajectories with 3 parameters (drift, bound, starting point) using Euler Mayurama  method
	- Obtain histograms and psychometrics
	- Compare to analytic solutions, explore the error of the approximation with dt
	- Explore dependence of histograms and psychometrics with parameter values
- Add time varying bound, variability in parameters
- Explore race models

- "Optogenetic manipulations"
	- "Sensory neurons" vs "Integrator neurons"
	- Bound "deletion"
        - Other ways?

- History effects
	- Explore updates on starting point and drifts
	- Also post error slowing (increase bounds after errors)

### Basic setup

In [ ]:
# Define DDM parameters
mu = 0.5 # drift
theta = 1 # bound
z = 0 # starting point
sigma = 1 # noise std

# Define simulation parameters
n_trials = 1000
dt = .001
T = 10

# time array and prealocate results
t = np.arange(0, T, dt)
n_t = t.size
S = -np.ones((n_trials,2))

In [ ]:
# Simulate trajectories

for tr in range(n_trials):
    
    dW   = np.random.randn(n_t,1) # noise vector
    dx   = mu*dt + sigma*dW*np.sqrt(dt)
    x    = np.cumsum(dx,0)
    
    for ti in range(n_t):
             
        if x[ti] >= theta:
           S[tr,:] = [1, t[ti]]                                         
           break
        elif x[ti] <= -theta:
           S[tr,:] = [0, t[ti]]         
           break

# Get the hit and error trials
hits = np.where(S[:,0]==1)
errs = np.where(S[:,0]==0)

In [ ]:
# Plot the reaction time distributions for the hit and error trials
hits_hist, hits_bin_edges = np.histogram(S[hits,1])
errs_hist, errs_bin_edges = np.histogram(S[errs,1])

plt.figure(figsize=(10,5))
plt.stairs(hits_hist, hits_bin_edges, label='hits')
plt.stairs(errs_hist, errs_bin_edges, label='errors')
plt.xlabel('Reaction time')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.show()

Expand next cell to see solution

In [ ]:
import numpy as np

# Define DDM parameters
mu = 0.5 # drift
theta = 1 # bound
z = 0 # starting point
sigma = 1 # noise std

# Define simulation parameters
n_trials = 1000
dt = .001
T = 10

# time array and prealocate results
t = np.arange(0, T, dt)
n_t = t.size
S = -np.ones((n_trials,2))

# Simulate trajectories

for j in range(n_trials):
    
    dW   = np.random.randn(n_t,1)
    dx   = dW*np.sqrt(dt) + mu*dt
    x    = np.cumsum(dx,0)
    
    for i in range(n_t):
             
        if (x[i] >= theta):
           S[j,:] = [1, t[i]]                                         
           break
        elif (x[i] <= -theta):
           S[j,:] = [0, t[i]]         
           break
    